In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline

Importing Relevant Data only from set. The CPT4, or prodedure code, for an appendectomy is 44960 or 44950. To narrow down the 6 GB of data to one particular condition, use grep in terminal. First use:
#head -1 Claim_Medical_Line.csv > appendectomy_44960.csv

to preserve the column labels
then use:

#grep 44960 Claim_Medical_Line.csv >> appendectomy_44960.csv 

to add the relevant claims data with that CPT4. 
Finally, to merge in the additional appendenctomy data that uses the other CPT4 code:

#grep 44950 Claim_Medical_line.csv >> appendectomy_44960.csv



In [9]:
claims_data_df = pd.read_csv('appendectomy_44960.csv', index_col=[0])
claims_data_df

,Claim_Line_No,Place_of_Service_Cd,Rev_Cd,CPT4_ID,CPT4_Mod1_Cd,CPT4_Mod2_Cd,CPT4_Mod3_Cd,CPT4_Mod4_Cd,Service_Start_Dt,Units,...,NDC_Cd,Capitation_Flag,ER_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt,Dental_Flag,Unit_Of_Measure_Cd,Unit_Of_Measure_Desc,Service_Start_Year
Claim_ID,,,,,,,,,,,,,,,,,,,,,
171144960,1,11,NaN,90834,,,,,2016-02-03,1,...,0,N,N,0.00,0.00,69.00,N,UN,Units,2016
306644960,1,11,NaN,97014,,,,,2016-06-06,1,...,0,N,N,15.56,2.85,0.00,N,UN,Units,2016
306644960,2,11,NaN,97035,,,,,2016-06-06,1,...,0,N,N,12.67,2.67,0.00,N,UN,Units,2016
306644960,3,11,NaN,97110,,,,,2016-06-06,1,...,0,N,N,6.77,6.37,0.00,N,UN,Units,2016
306644960,4,11,NaN,97140,,,,,2016-06-06,1,...,0,N,N,0.00,5.97,0.00,N,UN,Units,2016
168044960,1,11,NaN,93271,,,,,2016-03-09,1,...,0,N,N,0.00,0.00,0.00,N,UN,Units,2016
167944960,1,11,NaN,97110,GP,,,,2016-05-25,2,...,0,N,N,15.00,0.00,0.00,N,UN,Units,2016
167944960,2,11,NaN,97140,GP,,,,2016-05-25,1,...,0,N,N,0.00,0.00,0.00,N,UN,Units,2016
167944960,3,11,NaN,97014,GP,,,,2016-05-25,1,...,0,N,N,0.00,0.00,0.00,N,UN,Units,2016


Looks like there's quite a few claims that don't match the expected CPT4 codes- 44960 and 44950. These are claims that have the five digit CPT4 code in their claims number and they should not be included in the data set. 

In [10]:
#Confirm that there are more than the expected number of CPT4 codes
claims_data_df['CPT4_ID'].nunique()

586

In [25]:
#Narrow down the DataFrame to the data points we care about
claims_data_df
claims_data_df1 = claims_data_df[claims_data_df['CPT4_ID'] == ('44950')]
claims_data_df1

,Claim_Line_No,Place_of_Service_Cd,Rev_Cd,CPT4_ID,CPT4_Mod1_Cd,CPT4_Mod2_Cd,CPT4_Mod3_Cd,CPT4_Mod4_Cd,Service_Start_Dt,Units,...,NDC_Cd,Capitation_Flag,ER_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt,Dental_Flag,Unit_Of_Measure_Cd,Unit_Of_Measure_Desc,Service_Start_Year
Claim_ID,,,,,,,,,,,,,,,,,,,,,
167990802,2,22,NaN,44950,,,,,2016-11-19,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
167993273,17,99,360.0,44950,,,,,2016-11-19,1,...,0,N,N,0.0,1743.96,0.00,N,UN,Units,2016
306700165,2,21,NaN,44950,22,,,,2016-07-04,1,...,0,N,N,0.0,130.74,0.00,N,UN,Units,2016
161137448,1,22,NaN,44950,,,,,2016-12-18,1,...,0,N,N,0.0,539.56,0.00,N,UN,Units,2016
156535710,10,99,360.0,44950,,,,,2016-08-06,1,...,0,N,N,0.0,0.00,250.04,N,UN,Units,2016
156534568,1,22,NaN,44950,,,,,2016-08-06,1,...,0,N,N,0.0,0.00,864.85,N,UN,Units,2016
148366057,1,21,NaN,44950,,,,,2016-04-13,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
145250203,1,21,NaN,44950,,,,,2016-05-14,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
340503131,1,21,NaN,44950,22,,,,2016-11-24,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016


In [26]:
claims_data_df2 = claims_data_df[claims_data_df['CPT4_ID'] == ('44960')]
claims_data_df2

,Claim_Line_No,Place_of_Service_Cd,Rev_Cd,CPT4_ID,CPT4_Mod1_Cd,CPT4_Mod2_Cd,CPT4_Mod3_Cd,CPT4_Mod4_Cd,Service_Start_Dt,Units,...,NDC_Cd,Capitation_Flag,ER_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt,Dental_Flag,Unit_Of_Measure_Cd,Unit_Of_Measure_Desc,Service_Start_Year
Claim_ID,,,,,,,,,,,,,,,,,,,,,
160760625,1,21,NaN,44960,,,,,2016-03-16,1,...,0,N,N,0.0,139.49,720.00,N,UN,Units,2016
155256678,2,11,NaN,44960,22,,,,2016-01-24,1,...,0,N,N,0.0,38.96,1156.43,N,UN,Units,2016
148338682,1,22,NaN,44960,80,,,,2016-04-11,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
148338681,1,22,NaN,44960,22,,,,2016-04-11,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
141417773,1,21,NaN,44960,,,,,2016-01-12,1,...,0,N,N,0.0,0.00,1116.64,N,UN,Units,2016
140790969,1,22,NaN,44960,,,,,2016-01-08,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
140308027,2,21,NaN,44960,,,,,2016-02-01,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
331528134,1,21,NaN,44960,,,,,2016-02-21,1,...,0,N,N,0.0,162.08,0.00,N,UN,Units,2016
331528134,2,21,NaN,44960,,,,,2016-02-21,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016


In [27]:
#merge the two relevant DataFrames for 44950 and 44960
merged_dataframes = [claims_data_df1, claims_data_df2]
appendectomy_df= pd.concat(merged_dataframes)
appendectomy_df

,Claim_Line_No,Place_of_Service_Cd,Rev_Cd,CPT4_ID,CPT4_Mod1_Cd,CPT4_Mod2_Cd,CPT4_Mod3_Cd,CPT4_Mod4_Cd,Service_Start_Dt,Units,...,NDC_Cd,Capitation_Flag,ER_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt,Dental_Flag,Unit_Of_Measure_Cd,Unit_Of_Measure_Desc,Service_Start_Year
Claim_ID,,,,,,,,,,,,,,,,,,,,,
167990802,2,22,NaN,44950,,,,,2016-11-19,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
167993273,17,99,360.0,44950,,,,,2016-11-19,1,...,0,N,N,0.0,1743.96,0.00,N,UN,Units,2016
306700165,2,21,NaN,44950,22,,,,2016-07-04,1,...,0,N,N,0.0,130.74,0.00,N,UN,Units,2016
161137448,1,22,NaN,44950,,,,,2016-12-18,1,...,0,N,N,0.0,539.56,0.00,N,UN,Units,2016
156535710,10,99,360.0,44950,,,,,2016-08-06,1,...,0,N,N,0.0,0.00,250.04,N,UN,Units,2016
156534568,1,22,NaN,44950,,,,,2016-08-06,1,...,0,N,N,0.0,0.00,864.85,N,UN,Units,2016
148366057,1,21,NaN,44950,,,,,2016-04-13,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
145250203,1,21,NaN,44950,,,,,2016-05-14,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
340503131,1,21,NaN,44950,22,,,,2016-11-24,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016


Looks like this is a really small dataset. Let's look for a different procedure that might have more data
#MOAR DATA

In [28]:
caesarian_df = pd.read_csv('caesarian_data.csv', index_col=[0])
caesarian_df

,Claim_Line_No,Place_of_Service_Cd,Rev_Cd,CPT4_ID,CPT4_Mod1_Cd,CPT4_Mod2_Cd,CPT4_Mod3_Cd,CPT4_Mod4_Cd,Service_Start_Dt,Units,...,NDC_Cd,Capitation_Flag,ER_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt,Dental_Flag,Unit_Of_Measure_Cd,Unit_Of_Measure_Desc,Service_Start_Year
Claim_ID,,,,,,,,,,,,,,,,,,,,,
340993570,1,21,NaN,59510,,,,,2016-07-29,1,...,0,N,N,0.0,715.54,715.54,N,UN,Units,2016
350588632,1,21,NaN,59510,,,,,2016-06-27,1,...,0,N,N,0.0,641.99,0.00,N,UN,Units,2016
357047267,1,21,NaN,59510,UC,,,,2016-10-17,1,...,0,Y,N,0.0,0.00,0.00,N,UN,Units,2016
340423363,1,21,NaN,59510,,,,,2016-02-21,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
171159994,1,21,NaN,59510,,,,,2016-02-05,1,...,0,N,N,0.0,295.52,0.00,N,UN,Units,2016
171159994,2,21,NaN,59510,,,,,2016-02-05,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
171159510,1,22,NaN,G0202,26,,,,2016-02-03,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
171159510,2,22,NaN,77052,26,,,,2016-02-03,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
171175684,1,21,NaN,59510,,,,,2016-01-31,1,...,0,N,N,0.0,277.24,0.00,N,UN,Units,2016


Cleaning


In [29]:
caesarian_df['CPT4_ID'].nunique()

412

In [31]:
caesarian_df2 = caesarian_df[caesarian_df['CPT4_ID'] == ('59510')]
caesarian_df2

,Claim_Line_No,Place_of_Service_Cd,Rev_Cd,CPT4_ID,CPT4_Mod1_Cd,CPT4_Mod2_Cd,CPT4_Mod3_Cd,CPT4_Mod4_Cd,Service_Start_Dt,Units,...,NDC_Cd,Capitation_Flag,ER_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt,Dental_Flag,Unit_Of_Measure_Cd,Unit_Of_Measure_Desc,Service_Start_Year
Claim_ID,,,,,,,,,,,,,,,,,,,,,
340993570,1,21,NaN,59510,,,,,2016-07-29,1,...,0,N,N,0.0,715.54,715.54,N,UN,Units,2016
350588632,1,21,NaN,59510,,,,,2016-06-27,1,...,0,N,N,0.0,641.99,0.00,N,UN,Units,2016
357047267,1,21,NaN,59510,UC,,,,2016-10-17,1,...,0,Y,N,0.0,0.00,0.00,N,UN,Units,2016
340423363,1,21,NaN,59510,,,,,2016-02-21,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
171159994,1,21,NaN,59510,,,,,2016-02-05,1,...,0,N,N,0.0,295.52,0.00,N,UN,Units,2016
171159994,2,21,NaN,59510,,,,,2016-02-05,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
171175684,1,21,NaN,59510,,,,,2016-01-31,1,...,0,N,N,0.0,277.24,0.00,N,UN,Units,2016
171175684,2,21,NaN,59510,,,,,2016-01-31,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
171157745,1,99,NaN,59510,,,,,2016-02-08,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016


In [57]:
#Screen for columns that only have one value
caesarian_df2['Copay_Amt'].nunique()

7

In [46]:
caesarian_df2.isnull().sum(axis = 0)

Claim_Line_No              0
Place_of_Service_Cd        0
Rev_Cd                  6248
CPT4_ID                    0
CPT4_Mod1_Cd               0
CPT4_Mod2_Cd               0
CPT4_Mod3_Cd               0
CPT4_Mod4_Cd               0
Service_Start_Dt           0
Units                      0
Charge_Amt                 0
Prepaid_Amt                0
Plan_Paid_Amt              0
NDC_Cd                     0
Capitation_Flag            0
ER_Flag                    0
Copay_Amt                  0
Coinsurance_Amt            0
Deductible_Amt             0
Dental_Flag                0
Unit_Of_Measure_Cd         0
Unit_Of_Measure_Desc       0
Service_Start_Year         0
dtype: int64

In [51]:
#Remove Columns that are redundant (i.e. descriptive of another column in the table) or contain only one value
edited_caesarian_df = caesarian_df2.drop(['Dental_Flag', 'Units', 'Rev_Cd', 'NDC_Cd', 'ER_Flag', 'Service_Start_Year', 'Unit_Of_Measure_Desc', 'Unit_Of_Measure_Cd'], axis=1)
edited_caesarian_df

,Claim_Line_No,Place_of_Service_Cd,CPT4_ID,CPT4_Mod1_Cd,CPT4_Mod2_Cd,CPT4_Mod3_Cd,CPT4_Mod4_Cd,Service_Start_Dt,Charge_Amt,Prepaid_Amt,Plan_Paid_Amt,Capitation_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt
Claim_ID,,,,,,,,,,,,,,,
340993570,1,21,59510,,,,,2016-07-29,3350.00,0.00,1669.58,N,0.0,715.54,715.54
350588632,1,21,59510,,,,,2016-06-27,3500.00,0.00,688.25,N,0.0,641.99,0.00
357047267,1,21,59510,UC,,,,2016-10-17,3586.00,0.00,2029.14,Y,0.0,0.00,0.00
340423363,1,21,59510,,,,,2016-02-21,3800.00,1932.28,96.22,N,0.0,0.00,0.00
171159994,1,21,59510,,,,,2016-02-05,2689.03,0.00,1182.07,N,0.0,295.52,0.00
171159994,2,21,59510,,,,,2016-02-05,1160.97,0.00,1160.97,N,0.0,0.00,0.00
171175684,1,21,59510,,,,,2016-01-31,2710.84,0.00,1108.96,N,0.0,277.24,0.00
171175684,2,21,59510,,,,,2016-01-31,1089.16,0.00,1089.16,N,0.0,0.00,0.00
171157745,1,99,59510,,,,,2016-02-08,3850.00,0.00,2638.56,N,0.0,0.00,0.00


In [56]:
#There are few rows with CPT4 modifications. For the first model, drop them to simplify the process
simplified_CPT4_df = edited_caesarian_df.drop(['CPT4_Mod1_Cd', 'CPT4_Mod2_Cd', 'CPT4_Mod3_Cd', 'CPT4_Mod4_Cd'], axis=1)
simplified_CPT4_df

,Claim_Line_No,Place_of_Service_Cd,CPT4_ID,Service_Start_Dt,Charge_Amt,Prepaid_Amt,Plan_Paid_Amt,Capitation_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt
Claim_ID,,,,,,,,,,,
340993570,1,21,59510,2016-07-29,3350.00,0.00,1669.58,N,0.0,715.54,715.54
350588632,1,21,59510,2016-06-27,3500.00,0.00,688.25,N,0.0,641.99,0.00
357047267,1,21,59510,2016-10-17,3586.00,0.00,2029.14,Y,0.0,0.00,0.00
340423363,1,21,59510,2016-02-21,3800.00,1932.28,96.22,N,0.0,0.00,0.00
171159994,1,21,59510,2016-02-05,2689.03,0.00,1182.07,N,0.0,295.52,0.00
171159994,2,21,59510,2016-02-05,1160.97,0.00,1160.97,N,0.0,0.00,0.00
171175684,1,21,59510,2016-01-31,2710.84,0.00,1108.96,N,0.0,277.24,0.00
171175684,2,21,59510,2016-01-31,1089.16,0.00,1089.16,N,0.0,0.00,0.00
171157745,1,99,59510,2016-02-08,3850.00,0.00,2638.56,N,0.0,0.00,0.00


Feature Engineering To Do List
* Turn Capitation Y/N values into 1/0
* Make Date of Service a Time Stamp
* Turn Place of Service into a Categorical Feature (1-6)
* Add a column to turn the following into binary whether it's a numeric value or 0 to be used as a feature in linear regression:
    * Coinsurance
    * Prepaid
    * Deductible
    * CoPay
    

Engineering and Featurization 
Transform all of the non-numeric data into numeric data;
*get dummies
*masking
*boolean
*?

First Shitty Model
Initial Exploration of Features 


Tweakage of Model

Random Forest- Feature Importance/Decision Tree Evaluation

Quantile Regression- see Matt

Boosting-